# DDF Common Editor

This is the main colab that performs unit testing on ddf_common.
Unlike other colabs, you will always run this colab using google drive as the destination for a git clone/pull of ddf_common.
This ensures a permanent location of checked out files.

In [1]:
#@title DdfCommon Source Control Pane
from ipywidgets import interact_manual
from ipywidgets import Text, Checkbox
import os
import subprocess
import sys
import shutil

#@markdown Check out DdfCommon to leverage common imports!

#@markdown If you use a branch, you may change source files and check them in.

#@markdown ----
def ddf_commit_branch(commit_message: str, push: bool = False):
  print('executing commit_branch...')
  if (len(commit_message) == 0):
    print('No commit message provided.')
    return
  try:
    os.chdir(checked_out_path)
  except NameError:
    print(f"Branch not checked out for editing!")
    return

  os.system(f'git commit -m "{commit_message}"')
  print(f"commited change.")
  if push:
    os.system(f'git push')
    print(f"Pushed to branch {checked_out_branch}.")
  os.chdir("/content")

def ddf_commit_pane():
    interact_manual.options(manual_name="Commit Code")(
        ddf_commit_branch,
        commit_message = Text(value="",placeholder="Enter commit message",
             description="Commit Message",disabled=False),
        push=Checkbox(value=False, description="Push to " + checked_out_branch,disabled=False)
    )

def ddf_import_common(branch_name = ""):
  print(f'executing checkout_branch{branch_name}...')
  #branch_name = "" #@param {type:"string"}
  global checked_out_branch
  global checked_out_path
  try:
    print(f"Branch {checked_out_branch} already checked out.")
    print(f"You must restart the runtime to check out again.")
    ddf_commit_pane()
    return
  except NameError:
    checked_out_branch = branch_name if branch_name != "" else "main"

  if branch_name == "":
    shutil.rmtree( '/tmp/ddf_common' )
    os.chdir("/tmp")
    os.system(f'git clone --quiet https://github.com/tnc-br/ddf_common.git || echo "Repository already exists."')
    os.chdir("/content")
    checked_out_branch = "main"
    if '/tmp/ddf_common' not in sys.path:
      sys.path.insert(0,'/tmp/ddf_common')
    print('main branch checked out as readonly. You may now use ddf_common imports')
  else:
    if not os.path.isdir("/content/gdrive"):
      from google.colab import drive
      drive.mount("/content/gdrive")

    os.makedirs(f"/content/gdrive/MyDrive/{branch_name}")
    os.chdir(f"/content/gdrive/MyDrive/{branch_name}")
    os.system(f'git clone -b {branch_name} --quiet https://github.com/tnc-br/ddf_common.git || echo "Repository already exists."')
    os.chdir(f"/content/gdrive/MyDrive/{branch_name}/ddf_common")
    os.system(f'git pull')
    os.chdir(f"/content")
    checked_out_branch = branch_name
    checked_out_path = f'/content/gdrive/MyDrive/{branch_name}/ddf_common'
    if checked_out_path not in sys.path:
      sys.path.insert(0,checked_out_path)
    print(f'{checked_out_branch} branch checked out at "{checked_out_path}". You may now use ddf_common imports and change common files.')
    ddf_commit_pane()

def ddf_source_control_pane():
  interact_manual.options(manual_name="Checkout Code")(ddf_import_common,branch_name=Text(value="",placeholder="Enter branch name",description="Branch name",disabled=False));

if __name__ == "__main__":
  # to simply use ddf_common, call import_ddfcommon()
  # to change files, use the source_control_pane as below
  ddf_source_control_pane()


interactive(children=(Text(value='', description='Branch name', placeholder='Enter branch name'), Button(descr…